## 1. Import Package



In [1]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

d:\conda\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
d:\conda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
d:\conda\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24

In [ ]:
os.chdir("F:\Masters Data\Germany\Cottbus\3rd Semester\Market integration and sector coupling\BTU_lectures_2024\Lect_3")

In [6]:
# Read excel file which contains Non renewable generators data
bus=pd.read_excel('HW1.xlsx',sheet_name='buses',header=0)

In [7]:
bus

,bus,v_nom,carrier,x,y
0,North,380,AC,10.4515,51.1657
1,East,380,AC,2.2137,46.2276
2,West,380,AC,4.4699,50.5039


In [8]:
for index, row in bus.iterrows():
    network.add(
        "Bus",
        name=row['bus'],
        v_nom=row['v_nom'],
        carrier=row['carrier'],
        x=row['x'],
        y=row['y']
    )

In [9]:
network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
North,380.0,,10.4515,51.1657,AC,,1.0,0.0,inf,PQ,,
East,380.0,,2.2137,46.2276,AC,,1.0,0.0,inf,PQ,,
West,380.0,,4.4699,50.5039,AC,,1.0,0.0,inf,PQ,,


In [38]:
generators=pd.read_excel('HW1.xlsx',sheet_name='generators')

In [39]:
generators

,generator,p_nom,p_nom_max,bus,p_nom_extendable,marginal_cost,lifetime,efficiency,start_up_cost,shut_down_cost,min_up_time,min_down_time,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
0,NorthGen,31,31,North,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EastGen,19,19,East,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WestGen,10,10,West,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from pivottablejs import pivot_ui

In [41]:
# Step 1: Remove all generators
for generator_name in network.generators.index.tolist():
    network.remove("Generator", generator_name)


In [42]:
network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),
    p_nom_max=list(generators.p_nom_max),  # MW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

C:\Users\Mohammad Komeil\AppData\Local\Temp\ipykernel_25232\1087367996.py:1: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,e_sum_min,e_sum_max,q_set,sign,carrier,marginal_cost,marginal_cost_quadratic,active,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,stand_by_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NorthGen,North,PQ,,31.0,0.0,False,0.0,31.0,0.0,1.0,0.0,-inf,inf,0.0,1.0,NorthGen,1.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
EastGen,East,PQ,,19.0,0.0,False,0.0,19.0,0.0,1.0,0.0,-inf,inf,0.0,1.0,EastGen,1.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
WestGen,West,PQ,,10.0,0.0,False,0.0,10.0,0.0,1.0,0.0,-inf,inf,0.0,1.0,WestGen,1.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [14]:
# Read excel file which contains Non renewable generators data
load=pd.read_excel('HW1.xlsx',sheet_name='load',header=0)

In [16]:
load

,bus,load
0,North,10
1,East,10
2,West,40


In [17]:
for index, row in load.iterrows():
    network.add(
        "Load",
        name=row['bus'],
        bus=row['bus'],
        p_set=row['load'],
    )

In [18]:
network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
North,North,,,10.0,0.0,-1.0,True
East,East,,,10.0,0.0,-1.0,True
West,West,,,40.0,0.0,-1.0,True


In [31]:
# Read excel file which contains Non renewable generators data
lines=pd.read_excel('HW1.xlsx',sheet_name='lines',header=0)

In [32]:
lines.head()

,country_1,country_2,s_nom_extendable,s_nom
0,North,East,False,14
1,East,West,False,14
2,West,North,False,14


In [30]:
for line_name in network.lines.index.tolist():
    network.remove("Line", line_name)


In [33]:
# Add line connection between the buses
for index, row in lines.iterrows():
    network.add(
        "Line",
        "{}_{}".format(row['country_1'],row['country_2']),
        bus0=row['country_1'],
        bus1=row['country_2'],
        s_nom=row['s_nom'],
        x=1,
    )

In [34]:
network.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,active,build_year,lifetime,length,carrier,terrain_factor,num_parallel,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
North_East,North,East,,1.0,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
East_West,East,West,,1.0,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
West_North,West,North,,1.0,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
network

PyPSA Network
Components:
 - Bus: 3
 - Generator: 3
 - Line: 3
 - Load: 3
Snapshots: 1

In [43]:
network.optimize(solver_name='highs')

Index(['NorthGen', 'EastGen', 'WestGen'], dtype='object', name='Generator')
Index(['North', 'East', 'West'], dtype='object', name='Bus')
Index(['North_East', 'East_West', 'West_North'], dtype='object', name='Line')
Index(['North_East', 'East_West', 'West_North'], dtype='object', name='Line')
Index(['0'], dtype='object', name='SubNetwork')
Index(['NorthGen', 'EastGen', 'WestGen'], dtype='object', name='Generator')
Index(['North', 'East', 'West'], dtype='object', name='Bus')
Index(['North_East', 'East_West', 'West_North'], dtype='object', name='Line')
Index(['North_East', 'East_West', 'West_North'], dtype='object', name='Line')
Index(['0'], dtype='object', name='SubNetwork')
d:\conda\Lib\site-packages\linopy\common.py:154: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: So

('warning', 'infeasible')

In [44]:
network.pf()

{'n_iter': SubNetwork  0
 snapshot     
 0           2,
 'error': SubNetwork             0
 snapshot                
 0           1.509502e-10,
 'converged': SubNetwork     0
 snapshot        
 0           True}

In [47]:
network.buses_t

{'v_mag_pu_set': Empty DataFrame
 Columns: []
 Index: [0],
 'p': Bus       North  East  West
 snapshot                   
 0          50.0 -10.0 -40.0,
 'q': Bus          North  East  West
 snapshot                      
 0         0.009695   0.0   0.0,
 'v_mag_pu': Bus       North  East  West
 snapshot                   
 0           1.0   1.0   1.0,
 'v_ang': Bus       North      East      West
 snapshot                           
 0           0.0 -0.000139 -0.000208,
 'marginal_price': Empty DataFrame
 Columns: []
 Index: [0]}

In [48]:
network.generators_t

{'p_min_pu': Empty DataFrame
 Columns: []
 Index: [0],
 'p_max_pu': Empty DataFrame
 Columns: []
 Index: [0],
 'p_set': Empty DataFrame
 Columns: []
 Index: [0],
 'q_set': Empty DataFrame
 Columns: []
 Index: [0],
 'marginal_cost': Empty DataFrame
 Columns: []
 Index: [0],
 'marginal_cost_quadratic': Empty DataFrame
 Columns: []
 Index: [0],
 'efficiency': Empty DataFrame
 Columns: []
 Index: [0],
 'stand_by_cost': Empty DataFrame
 Columns: []
 Index: [0],
 'ramp_limit_up': Empty DataFrame
 Columns: []
 Index: [0],
 'ramp_limit_down': Empty DataFrame
 Columns: []
 Index: [0],
 'p': Generator  NorthGen  EastGen  WestGen
 snapshot                             
 0              60.0      0.0      0.0,
 'q': Generator  NorthGen  EastGen  WestGen
 snapshot                             
 0          0.009695      0.0      0.0,
 'status': Empty DataFrame
 Columns: []
 Index: [0],
 'mu_upper': Empty DataFrame
 Columns: []
 Index: [0],
 'mu_lower': Empty DataFrame
 Columns: []
 Index: [0],
 'mu_p_s

In [49]:
network.lines_t.p0

,North_East,East_West,West_North
snapshot,,,
0,20.0,10.0,-30.0
